# Cross validation and simple linear regression.

##### Predicting salaries of employees using linear regression

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels as sm
from statsmodels.tools.eval_measures import rmse
from scipy import stats
from scipy.stats import sem
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings('ignore')

salary_data = pd.read_csv('salary.csv')
salary_meta = pd.read_csv('Salary metadata.csv', sep=';')


In [2]:
display(salary_data)
display(salary_meta)


,salary,exprior,yearsworked,yearsrank,market,degree,otherqual,position,male,Field,yearsabs
0,53000.0,0,0,0,1.17,1,0,1,1,3,0
1,58000.0,1,0,0,1.24,1,0,1,1,2,0
2,45500.0,0,0,0,1.21,1,0,1,1,3,2
3,35782.0,0,2,1,0.99,1,0,1,1,4,1
4,34731.0,0,2,2,0.91,1,0,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
509,45906.0,6,17,11,0.92,1,0,2,0,2,70
510,60402.0,6,19,7,0.86,1,0,3,0,1,72
511,53187.0,19,7,6,0.78,1,0,3,0,4,76
512,56542.0,8,20,10,0.78,1,0,3,0,2,78


,Variable,Type,Explanation
0,salary,Continuous,Annual salary in dollars
1,exprior,Continuous,Years of experience prior to working in this f...
2,yearsworked,Continuous,Years worked in this field
3,yearsrank,Continuous,Years worked at current rank
4,market,Continuous,Market value (1 = salary at market value for p...
5,degree,Binary,"Has degree (0 = no, 1 = yes)"
6,otherqual,Binary,Has other post-secondary qualification (0 = no...
7,position,Categorical,"Position (1 = Junior Employee, 2 = Manager, 3 ..."
8,male,Binary,"0 = no, 1 = yes"
9,Field,Categorical,"Field of work (1 = Engineering, 2 = Finance, 3..."


## 1. Spliting data into a training and test set. Examining the training data for missing and extreme values. Creating histograms to show the distribution of the variables and a scatterplot showing the relationship between Years Worked and Salary.

#### Checking if there are any missing values

In [3]:
salary_data.isnull().sum()

salary         1
exprior        0
yearsworked    0
yearsrank      0
market         0
degree         0
otherqual      0
position       0
male           0
Field          0
yearsabs       0
dtype: int64

 one missing values from salary column

#### Filling the missing value with the mean

In [4]:
salary_data['salary'].fillna(salary_data['salary'].mean(),inplace=True)

In [5]:
# confirmation from missing values
salary_data.isnull().sum().sum()

0

##### Data correlation

In [6]:
salary_data.corr()

,salary,exprior,yearsworked,yearsrank,market,degree,otherqual,position,male,Field,yearsabs
salary,1.000000,0.119946,0.623135,0.610580,0.407208,0.044552,0.167129,0.701354,0.361447,-0.498485,-0.069622
exprior,0.119946,1.000000,-0.246549,-0.066403,-0.035041,-0.049373,0.264804,0.116596,0.046558,-0.048315,0.132380
yearsworked,0.623135,-0.246549,1.000000,0.813471,-0.070896,0.028421,-0.040490,0.746736,0.278963,-0.261379,0.055468
yearsrank,0.610580,-0.066403,0.813471,1.000000,-0.026975,0.006516,0.007036,0.485210,0.237787,-0.263953,0.035632
market,0.407208,-0.035041,-0.070896,-0.026975,1.000000,0.036408,0.021692,-0.013358,0.181201,-0.223827,-0.167068
degree,0.044552,-0.049373,0.028421,0.006516,0.036408,1.000000,-0.214717,0.043680,0.061611,-0.098424,0.029311
otherqual,0.167129,0.264804,-0.040490,0.007036,0.021692,-0.214717,1.000000,0.160311,0.015833,-0.076623,-0.017639
position,0.701354,0.116596,0.746736,0.485210,-0.013358,0.043680,0.160311,1.000000,0.318129,-0.288812,0.029751
male,0.361447,0.046558,0.278963,0.237787,0.181201,0.061611,0.015833,0.318129,1.000000,-0.128874,-0.622179
Field,-0.498485,-0.048315,-0.261379,-0.263953,-0.223827,-0.098424,-0.076623,-0.288812,-0.128874,1.000000,-0.000127


The columns Yearsworked, Yearsrank and Position have high correlation

In [7]:
 salary_data.describe()

,salary,exprior,yearsworked,yearsrank,market,degree,otherqual,position,male,Field,yearsabs
count,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000
mean,50863.220098,2.926070,12.852140,7.052529,0.948521,0.964981,0.044747,2.132296,0.750973,2.529183,6.982490
std,12672.762650,4.791397,9.444695,6.414771,0.149380,0.184008,0.206950,0.820075,0.432871,1.127420,16.873156
min,29000.000000,0.000000,0.000000,0.000000,0.710000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,40000.000000,0.000000,4.000000,2.000000,0.840000,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000
50%,50165.500000,0.500000,12.000000,5.000000,0.920000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000
75%,60306.250000,4.000000,22.000000,12.000000,1.020000,1.000000,0.000000,3.000000,1.000000,4.000000,2.000000
max,96156.000000,25.000000,41.000000,28.000000,1.330000,1.000000,1.000000,3.000000,1.000000,4.000000,118.000000


#### splitting the dataset

In [8]:
x = salary_data['yearsworked']
y = salary_data['salary']


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)

In [9]:
x_train.head()

347    27
3       2
210    21
35      0
472    14
Name: yearsworked, dtype: int64

In [10]:
# shape of training data set
x_train.shape

(359,)

In [12]:
# linear regression
stats.pearsonr(x_train,y_train)

(0.6091219465607669, 7.835542693045908e-38)

In [13]:

#plotting scatter plot
sns.regplot(x='yearsworked', y='salary', color='b', data= x_train, ci= None)
plt.title(" Relationship between Years Worked and Salary", fontsize= 15)
plt.show()

KeyError: 'yearsworked'

In [ ]:
#plotting histogram & subplotting
f, axes = plt.subplots(1, 2,figsize=(15, 5))
grid = plt.GridSpec(1, 2, wspace=0.5, hspace=0.3)
plt.subplot(grid[0, 0])
x_train['yearsworked'].hist()
plt.title('Years worked variance', fontsize= 15)
plt.xlabel('Years Worked')
plt.ylabel('Frequency')

plt.subplot(grid[0, 1])
y_train.hist()
plt.title('Salary Variance', fontsize= 15)
plt.xlabel('Salary')
plt.ylabel('Frequency')

In [ ]:
#creating model
x_train = sm.add_constant(x_train['yearsworked'])
lm = sm.OLS(y_train, x_train).fit()

### 2. The statsmodels package and the training data, running a simple linear regression for Salary with one predictor variable: Years Worked

In [ ]:
lm.summary()

#### Reporting the amount of variance explained (R^2) and significance value (p) 

In [ ]:
print('R-squared: ', lm.rsquared)


In [ ]:
print('P-value: ', lm.pvalues)


R-squared is 0.3672 this means is 37%, this indicates that the variables are closely related to each other.

A p-value weighs the strength of the evidence (i.e what the data tells you about the population). A value of <0.05 implies significance and that of <0.01 implies high significance. Therefore, our p-value being =0.000 implies high significance. Therefore we can conclude that it is statistically significan

## 3. What the unstandardized coefficient (B or ‘coef’ in statsmodels) tells about the relationship between Years Worked and Salary.

An unstandardized coefficient represents the amount of change in a dependent variable Y due to a change of 1 unit of independent variable X. So in our case is true the salary does increase with years worked when we look at our graphs above

## 4. 95% confidence intervals [0.025, 0.975] means:
Our coefficients fall within the confidence interval range. Although we dont have 100% certainty, we are 95% confident that the values are valid.

## 5. The expected salary for someone with 12 years’ work experience.

In [ ]:
lm.predict([1,12])

expected salary = 50503.90

## 6. The expected salary for someone with 80 years’ work experience.

In [ ]:
lm.predict([1,80])

expected salary for someone over 80 years = 106749.84

* It is very unlikely for a person to work for 80 years

## 7.  Other employee characteristics might influence their salary

According to the correlation tablt above in Question 1, years ranked and position occupied are moderatly - strongly correlated to salary recieved.

In [ ]:
rmse_train=rmse(y_train, lm.predict(x_train))
rmse_test=rmse(y_test, lm.predict(x_test))
print('The rmse  for the train set is :' ,rmse_train)
print('The rmse  for the test set is :',rmse_test)
print('the difference in RMSE between training and testing is:', rmse_test - rmse_train)